# Tweepyでいろいろ遊んでみる

## 特定クエリーのサーチ

過去ツイートに対してクエリーサーチを行う

In [116]:
import tweepy
import datetime
import re

from auth import (
    consumer_key,
    consumer_secret,
    access_token,
    access_token_secret
)
auth = tweepy.OAuthHandler(consumer_key,consumer_secret)
auth.set_access_token(access_token,access_token_secret)
api = tweepy.API(auth)

# print api.home_timeline()[0]
#api.update_status(status='hello,world!')
search_result = api.search(q='アナ雪')
for result in search_result:
    print result.text

神田沙也加さんの歌声なまら好き

神田さん出てるからアナ雪観たって言っても過言じゃないもん
RT @deepvalley: この後に放映される『アナと雪の女王』に対して、ディズニーそんな好きでも無いけど評判だから見てみるかな程度のオタクの皆さんへ。
『エルサ姉ちゃんの氷雪系能力者としての戦闘力の高さ』に注目すると、すごい勢いで楽しめるのでオススメです。マーベルでもいい…
RT @na_mako: お祭り男 勝生勇利とヴィクトル王子
※イッテQアナ雪トレス https://t.co/nd4jkYdqbT
RT @tealc_skywalker: アナ王女殺害未遂で逮捕されたハンスのその後。
①祖国サザンアイルズの馬小屋で強制労働を課せられるがエルサの放った雪玉が直撃(そして馬糞山に突っ込む)。
②強制労働から逃げ出しサンフランソウキョウで指名手配。
③更にその後ズートピアで店を…
RT @_reikaoozeki: カラオケでアナ雪歌うとこうなっちゃうの。👑 https://t.co/3joTPyYxJu
RT @tealc_skywalker: アナ王女殺害未遂で逮捕されたハンスのその後。
①祖国サザンアイルズの馬小屋で強制労働を課せられるがエルサの放った雪玉が直撃(そして馬糞山に突っ込む)。
②強制労働から逃げ出しサンフランソウキョウで指名手配。
③更にその後ズートピアで店を…
RT @C4Dbeginner: 『アナ雪』は主題歌「Let It Go」を「ありのままの私」と訳した日本語詞が話題だったけど、すごいと思うのはリズムやメロディの乗せ方だけじゃなくて、キャラの口の動きに合うよう英語詞と日本語詞の最後の母音を合わせてるんだよね。特にこの辺はヒップ…
RT @t_fist_of_fury: アナ雪も好きだけど、同じくらい好きなレリゴーの空耳、「えり子 左行こう」 https://t.co/KnME4WLAAK
RT @Johnnys_YURI_: アナ雪といえば『とびら開けて』の王子感爆発の光一さんと最後の死んだ目をしている剛くん https://t.co/ElipzAzs5H
RT @safo5049: ところでアナ雪見てる人にダイマしたいんだけどエルサと同じ世界で生きれそうなクソイケメン冬の妖精ジャックフロスト君が出るガーディアンズ面白いから見てほしい。ホ

## api.user_timelineで取得できるデータの形式を確認

In [46]:
for status in tweepy.Cursor(api.user_timeline).items():
    #print status.text.encode('utf-8'), status.created_at
    print status

Status(contributors=None, truncated=False, text=u'\u30a2\u30ca\u96ea\u3068\u3044\u3048\u3070\u7814\u7a76\u5ba4\u306e\u51fa\u3057\u7269\u7528\u306b\u5fb9\u591c\u3067\u4f5c\u3063\u305f\u30d1\u30ed\u30c7\u30a3\u30fc\u52d5\u753b\u601d\u3044\u51fa\u3059\u2026\u3042\u308c\u306f\u826f\u3044\u51fa\u6765\u3060\u3063\u305f\u3002\u76f4\u524d\u307e\u3067\u306f\u6b7b\u306c\u307b\u3069\u30b9\u30d9\u3063\u305f\u3089\u3069\u3046\u3057\u3088\u3046\u3068\u7dca\u5f35\u3067\u30b2\u30ed\u5410\u304d\u305d\u3046\u3060\u3063\u305f\u3051\u3069', is_quote_status=False, in_reply_to_status_id=None, id=838048917275066370, favorite_count=0, _api=<tweepy.api.API object at 0x103ae5e10>, author=User(follow_request_sent=False, has_extended_profile=False, profile_use_background_image=True, _json={u'follow_request_sent': False, u'has_extended_profile': False, u'profile_use_background_image': True, u'default_profile_image': False, u'id': 91967088, u'profile_background_image_url_https': u'https://pbs.twimg.com/profile_back

KeyboardInterrupt: 

# 自分の過去一週間のツイートを以下の形式で取得する

- ツイートした日（日本時間）
- ツイート内容（urlが貼付されたものに限定。ただしツイート本文から短縮urlは削除）
- 貼付されていた短縮urlのオリジナルのurl

In [136]:
# -*- coding: utf-8 -*-
import tweepy
import datetime
import re

from auth import (
    consumer_key,
    consumer_secret,
    access_token,
    access_token_secret
)
auth = tweepy.OAuthHandler(consumer_key,consumer_secret)
auth.set_access_token(access_token,access_token_secret)
api = tweepy.API(auth)

tweet_data = dict()
tweets_list = list()

yobi = ["Mon","Tue","Wed","Thu","Fri","Sat","Sun"]

for status in tweepy.Cursor(api.user_timeline).items():
    jp_time = status.created_at + datetime.timedelta(hours=9) #日本時間に直す

    today = datetime.datetime.now()
    week_age = today - datetime.timedelta(days=7) #実行日から一週間前の日付を出す
    
    # リツイートとリプを対象外にするためのマッチング処理
    pattern = r'^RT' # retweet
    pattern2 = r'^@' # reply
    pattern3 = r'https?://[A-Za-z0-9.-]*/[A-Za-z0-9.-]*' #urlにマッチ
    retweet_match = re.search(pattern,status.text)
    reply_match = re.search(pattern2,status.text)
    
    if week_age < jp_time < today: #直近1週間以内のtweetに限定
        if not retweet_match or reply_match: # Excluded retweet and reply
            if len([i['expanded_url'] for i in status.entities['urls']])>0: # urlが貼付されているtweetに限定
                print jp_time.strftime("%Y/%m/%d")+'('+yobi[jp_time.weekday()]+')'
                shorten_url_remove_text = re.sub(pattern3,'',status.text) # tweetに含まれるurlを削除
                #print shorten_url
                print shorten_url_remove_text
                print i['expanded_url']
                tweet_data = {'posted_date' : jp_time.strftime("%Y/%m/%d")+'('+yobi[jp_time.weekday()]+')',
                              'tweet': shorten_url_remove_text,
                              'url' : i['expanded_url']
                             }
                tweets_list.append(tweet_data)
    else:
        break

2017/03/05(Sun)
深層学習用自宅GPUが欲しいんや…｜NVIDIA、GeForce GTX 1080 Ti発表 価格は699ドル 1080より35%性能向上 1080は値下げ - Mogura VR 
http://www.moguravr.com/gtx1080ti/
2017/03/04(Sat)
実家のお祭り。素敵な感じ｜生野鉱山の町 軒先にひな人形 - NHK兵庫県のニュース   
http://www3.nhk.or.jp/lnews/kobe/2024147411.html
2017/03/04(Sat)
噂の地元のすごい映画館。いけないの無念すぎる… 
https://twitter.com/earthcinemas_hi/status/837689619147739137
2017/03/03(Fri)
怖いのでそろそろ手放す 
https://twitter.com/coincheckjp/status/837648048691920896
2017/03/03(Fri)
爆発しろ｜同棲を解消する 
http://anond.hatelabo.jp/20170302172738
2017/03/03(Fri)
ここでも劇場版saoが熱く語られている…w 
https://twitter.com/rebuildfm/status/837179524412395520
2017/03/02(Thu)
デロンギかっこいいけどたっかいんだよね…｜デロンギの全自動コーヒーマシン新モデル「エレッタ」、3月1日発売。大容量＆タッチセンサー式で便利に [T-SITE LIFESTYLE] 
http://top.tsite.jp/lifestyle/table/i/34417747/
2017/03/01(Wed)
これ愉快だったもんなー 
https://twitter.com/make_jp/status/836853166113243136
2017/03/01(Wed)
「ご存知ないのも無理はないですが」w
チャンバラと言う名の徹底抗戦の姿勢 
https://twitter.com/kzmakinocfs/status/836789447739891714
2017/03/01(Wed)
人工知能 研究者が守るべき倫理指針 学会がまとめる 